In [117]:
from fpdf import FPDF
import requests
import json


fpl_id = 2024013
current_gw = 26

def get(url):
    response = requests.get(url)
    return json.loads(response.content)

def get_data(fpl_id, current_gw):

    my_team_url = 'https://fantasy.premierleague.com/' +\
        'api/entry/{}/event/{}/picks/'.format(fpl_id, current_gw)
    my_team = get(my_team_url)
    return my_team

def create_report(fpl_id, current_gw):
    
    data = get_data(fpl_id, current_gw)
    
    
    pdf = FPDF('L', 'pt', 'A4')
    pdf.add_page()

    # Add background image
    pdf.image('base_image.png', 0, 0, w=842)

    # Font settings
    pdf.set_text_color(255, 255, 255)
    pdf.add_font('RadikalBold', '', 'RadikalBold.ttf', uni=True)
    pdf.set_font('RadikalBold', '', 16)
    pdf.set_font_size(12)
    
    # Print rank
    
    rank = data['entry_history']['overall_rank']
    score = data['entry_history']['total_points']
                                  
                                  
    pdf.set_xy(200, 185.5)
    pdf.cell(2, txt='De Bruynevirus')
    
    pdf.set_xy(200, 199.5)
    pdf.cell(2, txt=str(score))
    
    pdf.set_xy(200, 214)
    pdf.cell(2, txt=str(rank))
                                  
    
    
    
    pdf.output('report.pdf', 'F')



In [118]:
create_report(fpl_id, current_gw)

In [78]:
data = get_data()

In [110]:
data

{'active_chip': 'bboost',
 'automatic_subs': [],
 'entry_history': {'event': 26,
  'points': 106,
  'total_points': 1710,
  'rank': 323042,
  'rank_sort': 325184,
  'overall_rank': 59483,
  'bank': 2,
  'value': 1031,
  'event_transfers': 1,
  'event_transfers_cost': 0,
  'points_on_bench': 0},
 'picks': [{'element': 12,
   'position': 1,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 277,
   'position': 2,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 300,
   'position': 3,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 556,
   'position': 4,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 156,
   'position': 5,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': False},
  {'element': 302,
   'position': 6,
   'multiplier': 1,
   'is_captain': False,
   'is_vice_captain': True},
  {'element': 254,
   'position